# Module to scrape and store data in local storage as CSV

* Adaptation from: https://github.com/jorgeviz/depcdmx

Fields:
name, description, location, link, price, operation, rooms, bathrooms, construction (m2), terrain (m2)

## 1. Dependencies

In [17]:
# 
import requests
import statistics
import pandas as pd
from bs4 import BeautifulSoup
import datetime as dt
import os
from pprint import pprint as pp

## 2. Scraping Inmuebles 24

In [10]:
# Definig variables
_root = 'https://www.inmuebles24.com/'
_state = 'nuevo-leon' # Cases: distrito-federal, edo.-de-mexico, 
                      # queretaro-provincia, morelos-provincia, nuevo-leon, yucatan, 
                      # quintana-roo-provincia, jalisco, veracruz-provincia, puebla-provincia
_operation = 'venta' # Cases: venta, renta
_base_url = _root + "inmuebles-en-" + _operation + "-en-" + _state + "-pagina-{}.html"
user_agent = "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.62 Safari/537.36"
ddir = 'data/'

In [ ]:
# Print URL
_base_url

In [12]:
# Function that appends the information of each one of the pages
def save(depts):
    """ Append page data
        Params:
        -----
        depts : pd.Dataframe()
            Dataframe of Departments
    """
    
    # Read Existant file to append
    _fname = ddir + "{}/inmuebles24" + "-" + _state + "-" + _operation + ".csv"
    _fname = _fname.format(dt.date.today().isoformat())
    try:
        df = pd.read_csv(_fname, delimiter = ',')
    except:
        print('New file, creating folder..')
        try:
            os.mkdir(ddir + '{}'.format(dt.date.today().isoformat()))
            print('Created folder!')
        except:
            print('Folder exists already!')
        df = pd.DataFrame()
    
    # Append data
    print(depts.head(1).to_dict())
    try:
        if df.empty:
            depts.set_index(['name', 'location']).to_csv(_fname, sep=',')
            print('Correctly saved file: {}'.format(_fname))
        else:
            df = pd.concat([df, depts])
            df.set_index(['name', 'location']).to_csv(_fname, sep=',')
            print('Correctly saved file: {}'.format(_fname))
    except Exception as e:
        print(e)
        print('Could not save file: {}'.format(_fname))

In [13]:
# Function that scrape information from one page at a time
def scrape(content):
    """ Scrape all listings per page """
    columns = ['name',
               'description',
               'location',
               'link',
               'price',
               'operation',
               'rooms',
               'bathrooms',
               'construction (m2)',
               'terrain (m2)']

    data = pd.DataFrame(columns = columns)
    # Generate soup
    soup = BeautifulSoup(content, 'html.parser')
    # Get Characteristics
    for d in soup.find_all(class_ = "posting-card"):
        temp_dict = {}
        try:
            temp_dict['name'] = d.find(class_="posting-title").text.strip()
            temp_dict['description'] = d.find(class_="posting-description").text.strip()
            temp_dict['location'] = ' '.join([j.strip() for j in d.find(class_="posting-location").text.strip().split('\n')])
            temp_dict['link'] = d.find(class_="posting-title").find('a').get('href')
            temp_dict['price'] = d.find(class_="first-price").text.strip()
            temp_dict['operation'] = _operation
            for li in d.find(class_="main-features").findAll('li'):
                li = li.text.lower()
                if 'recámara' in li:
                    temp_dict['rooms'] = statistics.mean([int(s) for s in li.split() if s.isdigit()])
                elif 'baño' in li:
                    temp_dict['bathrooms'] = statistics.mean([int(s) for s in li.split() if s.isdigit()])
                elif 'construido' in li:
                    temp_dict['construction (m2)'] = statistics.mean([int(s) for s in li.split() if s.isdigit()])
                elif 'terreno' in li:
                    temp_dict['terrain (m2)'] = statistics.mean([int(s) for s in li.split() if s.isdigit()])
        except Exception as e:
            print(e)
            continue
        data = data.append(temp_dict, ignore_index = True)
    print('Found {} depts'.format(len(data['name'])))
    return data

In [14]:
# Loop over pages to retrieve all info available
def paginate():
    """ Loop over pages to retrieve all info available
        Returns:
        -----
        pg_nums : int
            Number of pages scraped
    """
    pg_nums = 1
    while True:
        try:
            print(_base_url.format(pg_nums))
            r = requests.get(_base_url.format(pg_nums),
                             headers={'user-agent': user_agent})
            if r.status_code != 200:
                raise Exception("Wrong Response")
            depts = scrape(r.content) ## Start SCRAPE function
            if depts.empty:
                raise Exception("No more departments")
        except Exception as e:
            print(e)
            print('Finishing to retrieve info.')
            break
        
        # Store values
        save(depts) ## Save DATA into CSV
        pg_nums += 1
    return pg_nums

In [16]:
# Main method
print('Starting to scrape Inmuebles24')
paginate()

In [17]:
paginate()

https://www.inmuebles24.com/inmuebles-en-venta-en-nuevo-leon-pagina-1.html
Found 20 depts
{'name': {0: 'Lanka Huasteca | Pre - Venta ¡Enganche Desde 5%!'}, 'description': {0: 'Lanka significa “lugar que brilla”, y este desarrollo lo hace con luz propia ya que combina varios factores: la impecable arquitectura a cargo de Rodrigo de la Peña que aprovecha el entorno, espacios comunes originales que aportan plusvalía a la vivienda, y una ubicación privilegiada que mantiene el contacto con la naturaleza. Seguridad: Circuito Cerrado de Vigilancia 24/ 7, Control de acceso con tarjetas inteligentes, caseta de vigilancia. Amenidades al exterior: Más de 7, 000M2 con juegos infantiles, asadores, alberca, bosque de lectura, área de camping, cancha polivalente y más. Accesos: Accesos fáciles y rápidos desde Morones Prieto (a 1 minuto de Morones Prieto) Ubicación: a 3min del Consulado Americano. En frente del Sierra Madre Institute. Conveniencia: Complejo comercial al exterior con servicios. Enganch

Found 20 depts
{'name': {0: 'Oficinas en Torre Albia en Renta, Monterrey, Nuevo Leon'}, 'description': {0: 'Torre Albia. Oficina 126 mts. Edificio nuevo. Oficina a estrenar. Piso 7. Cuenta con cinco cajones de estacionamiento. - Obra Gris $41, 312. 00. - Obra Básica $58, 910. 00. - Obra Acondicionada $73, 380. 00. Nota: Precios + iva + Mantenimiento y sujetos a cambio sin previo aviso. Edificio aaa. Amenidades: Bussiness Center con 6 Salas de juntas con espacios privados de diferentes capacidades, Lobby doble altura con servicio de recepción y acceso controlado, terraza en planta baja, 5 elevadores de alta velocidad Cafetería para empleados, vigilancia 24 Hrs.'}, 'location': {0: 'Blvd. Antonio L. Rodríguez 3000, Santa María, Monterrey'}, 'link': {0: '/propiedades/oficinas-en-torre-albia-en-renta-monterrey-nuevo-58433804.html'}, 'price': {0: 'MN 6,009,000'}, 'operation': {0: 'venta'}, 'rooms': {0: nan}, 'bathrooms': {0: 0}, 'construction (m2)': {0: 126}, 'terrain (m2)': {0: 126}}
Correc

Found 20 depts
{'name': {0: 'Departamento Centro Monterrey 72 m $2,290,000 con Acabados Zavala y Treviño'}, 'description': {0: 'Excelente proyecto de 180 Departamentos de Lujo ubicados en el corazón de Monterrey. Sobre Jerónimo Treviño y Martin de Zavala. Los departamentos se entregan con acabados! Incluye puertas, ventanas, pintura blanca, pisos en concreto pulido, muebles de baño (lavabo, sanitario, llaves y regaderas) Amenidades Alberca, salon de usos multiples, ludoteca. 5 salas lounge, pet zone, 4 asadores. Gym, area de ejercicio al aire libre, seguridad 24/7. 2 terras con vistas panorámicas, 2 Tinas de Hidromasaje. Promoción y precios vigentes solo hasta febrero Y marzo 2019! El proyecto se entrega en septiembre 2020. Tipos de Departamentos: 72 m2 desde $2, 290, 000 2 recamaras, 2 baños. 96m2 desde $2, 950, 000 2 o 3 recamaras, 2 baños. forma de pago. Tradicional: 20% de enganche a meses sin intereses y 80% contra entrega. Construcción 20% anticipo, 70% a meses sin intereses y 10

Found 20 depts
{'name': {0: 'Casa en Venta Villa Sol Zona Sur Monterrey'}, 'description': {0: 'Excelente propiedad ubicada A pocos metros de la uanl (unidad mederos) . En planta baja cuenta con cochera para tres autos, recibidor, sala, comedor, cocina integral, lavanderia, 1 baño completo para visitas, dos recamaras (una de ellas con baño completo) , las dos recamaras cuentan con closet Y cuenta tambien con un espacio abierto que pudiera ser la tercera recamara. En la segunda planta cuenta con dos departamentos con entrada independiente cada uno. - el primer departamento cuenta con sala, comedor, cocineta con estufa, una recamara con baño completo Y closet, minisplit Y abanicos de techo. El segundo departamento cuenta con dos recamaras Y un baño completo mas un medio baño, sala, comedor, cocineta con estufa, minisplit Y abanicos de techo. (lv)'}, 'location': {0: 'Villa Sol, Monterrey'}, 'link': {0: '/propiedades/casa-en-venta-villa-sol-zona-sur-monterrey-57914913.html'}, 'price': {0: '

Found 20 depts
{'name': {0: 'Casa en Venta en Monterrey'}, 'description': {0: 'Hermosa casa en esquina en colonia privada con ubicación privilegiada y vigilancia 24/7. La casa cuenta con 2 plantas, sala, comedor, cocina integral con estufa y campana, 2 recámaras, 1. 5 baños, espacio de cochera para 2 autos y excedente de terreno. Es una excelente opción, le apoyamos con el trámite de su crédito.'}, 'location': {0: 'Arcos del Sol , Monterrey'}, 'link': {0: '/propiedades/casa-en-venta-en-monterrey-57865439.html'}, 'price': {0: 'MN 1,150,000'}, 'operation': {0: 'venta'}, 'rooms': {0: 2}, 'bathrooms': {0: 1}, 'construction (m2)': {0: 90}, 'terrain (m2)': {0: 120}}
Correctly saved file: data/2020-05-29/inmuebles24-nuevo-leon-venta.csv
https://www.inmuebles24.com/inmuebles-en-venta-en-nuevo-leon-pagina-27.html
Found 20 depts
{'name': {0: 'Casa en Venta - Santa Engracia/valle Ote. - San Pedro Garza Garcia'}, 'description': {0: 'Excelente casa nueva de reconocida firma de arquitectos. Doble Sa

...

Correctly saved file: data/2020-05-29/inmuebles24-nuevo-leon-venta.csv
https://www.inmuebles24.com/inmuebles-en-venta-en-nuevo-leon-pagina-1136.html
Found 20 depts
{'name': {0: 'Lanka Huasteca | Pre - Venta ¡Enganche Desde 5%!'}, 'description': {0: 'Lanka significa “lugar que brilla”, y este desarrollo lo hace con luz propia ya que combina varios factores: la impecable arquitectura a cargo de Rodrigo de la Peña que aprovecha el entorno, espacios comunes originales que aportan plusvalía a la vivienda, y una ubicación privilegiada que mantiene el contacto con la naturaleza. Seguridad: Circuito Cerrado de Vigilancia 24/ 7, Control de acceso con tarjetas inteligentes, caseta de vigilancia. Amenidades al exterior: Más de 7, 000M2 con juegos infantiles, asadores, alberca, bosque de lectura, área de camping, cancha polivalente y más. Accesos: Accesos fáciles y rápidos desde Morones Prieto (a 1 minuto de Morones Prieto) Ubicación: a 3min del Consulado Americano. En frente del Sierra Madre Inst

Correctly saved file: data/2020-05-29/inmuebles24-nuevo-leon-venta.csv
https://www.inmuebles24.com/inmuebles-en-venta-en-nuevo-leon-pagina-1142.html
Found 20 depts
{'name': {0: 'Lanka Huasteca | Pre - Venta ¡Enganche Desde 5%!'}, 'description': {0: 'Lanka significa “lugar que brilla”, y este desarrollo lo hace con luz propia ya que combina varios factores: la impecable arquitectura a cargo de Rodrigo de la Peña que aprovecha el entorno, espacios comunes originales que aportan plusvalía a la vivienda, y una ubicación privilegiada que mantiene el contacto con la naturaleza. Seguridad: Circuito Cerrado de Vigilancia 24/ 7, Control de acceso con tarjetas inteligentes, caseta de vigilancia. Amenidades al exterior: Más de 7, 000M2 con juegos infantiles, asadores, alberca, bosque de lectura, área de camping, cancha polivalente y más. Accesos: Accesos fáciles y rápidos desde Morones Prieto (a 1 minuto de Morones Prieto) Ubicación: a 3min del Consulado Americano. En frente del Sierra Madre Inst

Correctly saved file: data/2020-05-29/inmuebles24-nuevo-leon-venta.csv
https://www.inmuebles24.com/inmuebles-en-venta-en-nuevo-leon-pagina-1148.html
Found 20 depts
{'name': {0: 'Lanka Huasteca | Pre - Venta ¡Enganche Desde 5%!'}, 'description': {0: 'Lanka significa “lugar que brilla”, y este desarrollo lo hace con luz propia ya que combina varios factores: la impecable arquitectura a cargo de Rodrigo de la Peña que aprovecha el entorno, espacios comunes originales que aportan plusvalía a la vivienda, y una ubicación privilegiada que mantiene el contacto con la naturaleza. Seguridad: Circuito Cerrado de Vigilancia 24/ 7, Control de acceso con tarjetas inteligentes, caseta de vigilancia. Amenidades al exterior: Más de 7, 000M2 con juegos infantiles, asadores, alberca, bosque de lectura, área de camping, cancha polivalente y más. Accesos: Accesos fáciles y rápidos desde Morones Prieto (a 1 minuto de Morones Prieto) Ubicación: a 3min del Consulado Americano. En frente del Sierra Madre Inst

Correctly saved file: data/2020-05-29/inmuebles24-nuevo-leon-venta.csv
https://www.inmuebles24.com/inmuebles-en-venta-en-nuevo-leon-pagina-1154.html


KeyboardInterrupt: 

## 3. Scraping Segunda Mano

In [34]:
# Definig variables
_root = 'https://www.segundamano.mx/'
_state = 'aguascalientes' # Cases: 
_operation = 'venta' # Cases: venta, renta

_base_url = _root + 'anuncios/' + _state + '/' + _operation + '-inmuebles?page={}'
user_agent = "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.62 Safari/537.36"
ddir='data/'

In [35]:
# Print
_base_url

'https://www.segundamano.mx/anuncios/aguascalientes/venta-inmuebles?page={}'

In [36]:
# Function that appends the information of each one of the pages
def save(depts):
    """ Append page data
        Params:
        -----
        depts : list
            List of Departments
    """
    # Read Existant file to append
    _fname = ddir+"{}/segundamano.csv".format(dt.date.today().isoformat())
    try:
        df = pd.read_csv(_fname, delimiter='~')
    except:
        print('New file, creating folder..')
        try:
            os.mkdir(ddir+'{}'.format(dt.date.today().isoformat()))
            print('Created folder!')
        except:
            print('Folder exists already!')
        df = pd.DataFrame()
    # Append data
    depdf = pd.DataFrame(depts)
    print(depdf.head(1).to_dict())
    try:
        if df.empty:
            depdf.set_index(['name','location']).to_csv(_fname, sep='~')
            print('Correctly saved file: {}'.format(_fname))
        else:
            df = pd.concat([df, depdf])
            df.set_index(['name','location']).to_csv(_fname, sep='~')
            print('Correctly saved file: {}'.format(_fname))
    except Exception as e:
        print(e)
        print('Could not save file: {}'.format(_fname))

In [37]:
# Function that scrape information from one page at a time
def scrape(content):
    """ Scrape all departments per page
    """
    data = []
    # Generate soup
    soup = BeautifulSoup(content, 'html.parser')
    with open(ddir+'segundamano.html', 'w') as _F:
        _F.write(soup.prettify())
    # Get Characteristics
    for d in soup.find_all(class_="ad"):
        print('----')
        try:
            print(d) 
        except Exception as e:
            print(e)
            continue
        break
    print('Found {} depts'.format(len(data)))
    return data

In [38]:
# Loop over pages to retrieve all info available
def paginate():
    """ Loop over pages to retrieve all info available
        Returns:
        -----
        pg_nums : int
            Number of pages scraped
    """
    pg_nums = 1
    while True:
        try:
            print(_base_url.format(pg_nums))
            r = requests.get(_base_url.format(pg_nums),
                headers={'user-agent': user_agent})
            if r.status_code != 200:
                raise Exception("Wrong Response")
            depts = scrape(r.content) ## Start SCRAPE function
            if not depts:
                raise Exception("No more departments")
        except Exception as e:
            print(e)
            print('Finishing to retrieve info.')
            break
        # Store values
        save(depts) ## Save DATA into CSV
        pg_nums += 1
        break ###
    return pg_nums

In [39]:
# Main method
print('Starting to scrape Inmuebles24')
pages = paginate()

Starting to scrape Inmuebles24
https://www.segundamano.mx/anuncios/aguascalientes/venta-inmuebles?page=1
Found 0 depts
No more departments
Finishing to retrieve info.
